In [8]:
from sim import simulation as sim
import dask

In [4]:
%time sim.lorenz_96([1,2,3,4], 1)

CPU times: user 28 µs, sys: 4 µs, total: 32 µs
Wall time: 35 µs


array([-4., -2.,  4., -6.])

In [9]:
a = dask.delayed(sim.lorenz_96([1,2,3,4], 1))

AttributeError: module 'dask' has no attribute 'delayed'